## Step 1: Extract and Prepare the Dataset

In [8]:
# Notebook based on https://www.kaggle.com/code/tanvirnwu/simple-neural-network-training-imagenet

# Imports of this notebook
import os
import torch
import torchmetrics

import pandas as pd

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models

from PIL import Image
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader


# Set local Data directory for TinyImageneet
data_dir = './data/tiny-imagenet-200'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
print(train_dir, val_dir)


./data/tiny-imagenet-200/train ./data/tiny-imagenet-200/val


## Step 2: Define a Custom Dataset

In [9]:

"""Simple Tiny Imagenet Dataloader"""
class TinyImageNetDataset(Dataset):
    def __init__(self, root_dir, transform=None, train=True):
        self.root_dir = root_dir

        self.train = train
        self.transform = transform
        
        if self.train:
            self.data = []
            self.labels = []
            classes = sorted(os.listdir(os.path.join(root_dir, 'train')))
            
            # Search all images in disk and set the label as the index of the directory.
            for label, cls in enumerate(classes):
                cls_dir = os.path.join(root_dir, 'train', cls, 'images')
                for img_name in os.listdir(cls_dir):
                    self.data.append(os.path.join(cls_dir, img_name)) # store the path only
                    self.labels.append(label)
        else:
            self.data = []
            self.labels = []
            val_dir = os.path.join(root_dir, 'val', 'images')
            
            # ground-truth for validation data is stored on cvs files
            val_annotations = pd.read_csv(os.path.join(root_dir, 'val', 'val_annotations.txt'), 
                                          sep='\t', header=None, 
                                          names=['file_name', 'class', 'x1', 'y1', 'x2', 'y2']) 
            class_to_idx = {cls: idx for idx, cls in enumerate(sorted(os.listdir(os.path.join(root_dir, 'train'))))}
            for _, row in val_annotations.iterrows():
                self.data.append(os.path.join(val_dir, row['file_name'])) # again store the path only
                self.labels.append(class_to_idx[row['class']])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        # Load image and fetch labels
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        # Apply transform if available
        if self.transform:
            image = self.transform(image)
        
        return image, label


## Step 3 Instantiate Network, transformations and Loss function

In [10]:
# Defining the transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

# Loading the dataset
data_dir = data_dir
train_dataset = TinyImageNetDataset(root_dir=data_dir, transform=transform, train=True)
val_dataset = TinyImageNetDataset(root_dir=data_dir, transform=transform, train=False)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=4)

# Cuda Device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device is', device)

# Resnet18 imagente pretraining
model = models.resnet18(pretrained=False)

# Adapt the final layer for Tiny Imagenet
model.fc = torch.nn.Linear(model.fc.in_features, 200)

# Loss fucntion and optimizer
criterion = nn.CrossEntropyLoss()

device is cuda:0


/home/mb600l/conda_env/pytorch_GPU_3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mb600l/conda_env/pytorch_GPU_3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


## Step 4: Training Loop

In [11]:
# Training loop
model.to(device)

num_epochs = 10
train_losses = []
val_losses = []

optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    #for images, labels in train_loader:
    #for images, labels in tqdm(train_loader, desc="Training Progress", leave=True):
    with tqdm(train_loader, desc="Training", leave=True) as pbar:
        for batch_idx, (images, labels) in enumerate(pbar):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            outputs = model(images)          
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            avg_loss = running_loss / (batch_idx + 1)
            pbar.set_postfix(loss=avg_loss)
            
        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

    model.eval()
    running_loss = 0.0
    accuracy_metric = torchmetrics.Accuracy(task="multiclass", num_classes=200).to(device)
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            
            # Update metric
            predictions = torch.argmax(outputs, dim=1)
            accuracy_metric.update(predictions, labels)
            
            loss = criterion(outputs, labels)
            running_loss += loss.item()
    val_loss = running_loss / len(val_loader)
    val_losses.append(val_loss)

    validation_accuracy = accuracy_metric.compute()
    print(f"Validation Accuracy: {validation_accuracy:.4f}")

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

Training:   0%|          | 0/196 [00:00<?, ?it/s]/home/mb600l/conda_env/pytorch_GPU_3.9/lib/python3.9/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608883701/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Training: 100%|██████████| 196/196 [00:19<00:00, 10.27it/s, loss=4.28]


Validation Accuracy: 0.1559
Epoch [1/10], Train Loss: 4.2811, Val Loss: 3.8404


Training: 100%|██████████| 196/196 [00:18<00:00, 10.50it/s, loss=3.41]


Validation Accuracy: 0.2217
Epoch [2/10], Train Loss: 3.4055, Val Loss: 3.4839


Training: 100%|██████████| 196/196 [00:18<00:00, 10.42it/s, loss=2.97]


...

Validation Accuracy: 0.2918
Epoch [8/10], Train Loss: 0.9443, Val Loss: 3.7727


Training: 100%|██████████| 196/196 [00:18<00:00, 10.85it/s, loss=0.532]


Validation Accuracy: 0.2904
Epoch [9/10], Train Loss: 0.5322, Val Loss: 3.9864


Training: 100%|██████████| 196/196 [00:18<00:00, 10.83it/s, loss=0.253]


Validation Accuracy: 0.3041
Epoch [10/10], Train Loss: 0.2531, Val Loss: 4.2051
